<a href="https://colab.research.google.com/github/jtbigdata/Precios_de_alquileres/blob/main/dashboard_kingscounty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

!pip install dash
import dash
from dash import Dash, html, dcc


#import dash_core_components as dcc
#import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objs as go

!pip install dash-bootstrap-components
import dash_bootstrap_components as dbc

#import pandas_datareader.data as web
import datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.2/221.2 kB 5.0 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/content/drive')

url=r'/content/drive/MyDrive/Colab Notebooks/Proyecto_KingCounty/dataframe_Kingscounty.csv'

df = pd.read_csv(url, sep=',')

Mounted at /content/drive


In [5]:
#=======Cargamos el dataframe=======================

#df = pd.read_csv("dataframe_Kingscounty.csv")

#----------------------------------------------------------
#------------Agrupamos algunas variables de manera conveniente
#----------------------------------------------------------
col_dis=['Calidad','Condicion','no_habitaciones','no_baños',
       'Habitacion_baño','no_pisos','Vistaalmar','Indice_vista']

col_con=['Precio','m2_habitables','m2_Tot','m2_snm',
         'm2_bnm','m2_15viviendas','m2_15lotes']

dic={'Tot_vs_Habitables':["m2_habitables","m2_Tot"],
     'bnm_vs_snm': ["m2_snm","m2_bnm"],
     '15lotes_vs_15viviendas':["m2_15viviendas","m2_15lotes"]
     }
keys_list=['Tot_vs_Habitables','bnm_vs_snm','15lotes_vs_15viviendas']

#print(df.columns)

In [6]:
#=========================app=======================================
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.SIMPLEX],
                meta_tags=[{'name': 'viewport',
                            'content': 'width=device-width, initial-scale=1.0'}]
                )

In [7]:
#===================layout==============================================

app.layout = dbc.Container([
    #-------------------FILA 1-----------------------------------------
    dbc.Row([
        dbc.Col(
            html.H1("Venta de viviendas en Kings-County, Usa.",
                            #className='text-center text-primary mb-4',
                           style={'fontSize': 40,
                          'fontFamily':'sans-serif',
                          'color':'RebeccaPurple',
                          'textAlign':'center'}),width=12)
            ]),
    dbc.Row(html.Div(), style={'height':'40px'}),
    #-------------------FILA 2------------------------------------------
    dbc.Row([
        dbc.Col([dcc.Dropdown(id='dropdown_1',options=col_dis,
                              value='Calidad',multi=False,
                              clearable=False,style={"width": "100%"},
                              placeholder="Variable cualitativas")
                ]),
        dbc.Col([dcc.Dropdown(id='dropdown_2',options=col_con,value='Precio',
                            multi=False,clearable=False,style={"width": "100%"},
                            placeholder="Variable continuas")

                ]),
            ]),
    dbc.Row(html.Div(), style={'height':'20px'}),
    #-------------------FILA 3-----------------------------------------------------
    dbc.Row(html.Div(), style={'height':'20px'}),
    dbc.Row([
            #----------------------------------------------------------------
        dbc.Col([dcc.Graph(id='graph_1', figure={},style={"width": "95%"}),
                ]),
        dbc.Col([dcc.Graph(id='graph_2',figure={},style={"width": "95%"}),
                ]),
            #------------------------------------------------------------------
            ]),
    #--------------------FILA 4-------------------------------------------------------
    dbc.Row([
        dbc.Row(html.Div(), style={'height':'20px'}),
        dbc.Row([dcc.Graph(id='graph_3')
                 ]),
        dbc.Row([dcc.RadioItems(['yr_construccion','Fecha'],id='radioitem_1',
                                value='Fecha',#inline=True,
                                style={"padding": "10.px", "max-width": "800px", #"margin": "auto"
                                       'margin-left':'20px'}
                                )
                 ])
    ]),
    #---------------------FILA 5--------------------------------------------------------------------
    dbc.Row([
        dbc.Row(html.Div(), style={'height':'20px'}),
        dbc.Row([dcc.Graph(id='graph_4')]),
        dbc.Row([dcc.RadioItems(keys_list,id='radioitem_2',value='Tot_vs_Habitables'#,inline=True
                                )
                 ])
    ]),
    #---------------------FILA 6--------------------------------------------------------------------
    #style={"width": "80%"}
    dbc.Row(html.Div(), style={'height':'20px'}),
    dbc.Row([
        html.Label('Mapas de las viviendas',
                   style={'fontSize': 25,
                          'fontFamily':'sans-serif',
                          'color':'RebeccaPurple',
                          'textAlign':'center'}),
        dbc.Row([dcc.Graph(id='graph_5')],style={'height':'500px'}),
        html.Label('Ingrese las características de la vivienda.',
                   style={'fontSize': 20,
                          'fontFamily':'sans-serif',
                          'color':'RebeccaPurple',
                          'textAlign':'center'}
                   ),
        dbc.Col([dcc.Dropdown(id='id_hab',options=sorted(df.no_habitaciones.unique()),
                                placeholder="Número de habitaciones",value=[0,1,2],multi=True)
                ]),
        dbc.Col([dcc.Dropdown(id='id_grade',options=sorted(df.Calidad.unique()),value=['Muy buena'],
                        placeholder="Calidad",multi=True)
                ]),
        dbc.Col([
            dcc.Dropdown(id='id_baños',options=sorted(df.no_baños.unique()),value=[2],
                        placeholder="Número de baños",multi=True)
                ]),
        dbc.Col(dcc.Dropdown(id='id_con',options=sorted(df.Condicion.unique()),value=['Muy buena'],
                        placeholder='Condición',multi=True)),
        dbc.Col(dcc.Dropdown(id='id_mar',options=sorted(df.Vistaalmar.unique()),value=['Si','No'],
                        placeholder='Vista al mar',multi=True)),
             ]),
    dbc.Row(html.Div(), style={'height':'5px'}),
    dbc.Row([
            dbc.Col(dcc.Input(id='id_input1',type='number',value=0,
                              #value=min(df.precio),
                              placeholder='Ingrese valor > '+str(min(df.Precio)),
                              step=100000)),
            #dbc.Col(),
            dbc.Col(dcc.Input(id='id_input2',type='number',#value=max(df.precio)
                              value=7500000,
                              placeholder='Ingrese valor < '+str(max(df.Precio))
                              ,step=500000)),
            dbc.Col(),
            dbc.Col(),
            dbc.Col()
            ]),
    ], fluid=True)

In [8]:
#========CALLBACK==================================================

#--------------------------Callback 1 funnel---------------------
@app.callback(
    Output(component_id='graph_1', component_property='figure'),
    Input(component_id='dropdown_1', component_property='value'),
    Input(component_id='dropdown_2', component_property='value')
)

def update_graph(dropdown_1,dropdown_2):
    Xdf = df.groupby(dropdown_1)[dropdown_2].mean().reset_index()
    Xdf = Xdf.sort_values(by=dropdown_2, ascending=False)
    fig = go.Figure(go.Funnel(x = Xdf[dropdown_2],y = Xdf[dropdown_1]))

    fig.update_layout(
    title=str(dropdown_1)+' vs '+str(dropdown_2),
    title_x=0.5,
    #xaxis_title=str(dropdown_2),
    #yaxis_title=str(dropdown_1),
    legend_title=str(dropdown_1)+'vs'+str(dropdown_2),
    font=dict(
        family="sans-serif",
        size=18,
        color="RebeccaPurple"
    ),
    plot_bgcolor='rgba(245, 238, 248,1.0)',
    paper_bgcolor='white'
    )
    return (fig)

#------------------------Callback 2 violines-------------------------
@app.callback(
    Output(component_id='graph_2', component_property='figure'),
    Input(component_id='dropdown_1', component_property='value'),
    Input(component_id='dropdown_2', component_property='value')

)

def update_graph(dropdown_1 , dropdown_2):
    Xdf=df
    Xdf = df.sort_values(by=dropdown_2, ascending=False)
    fig = px.violin(Xdf, x=dropdown_1, y=dropdown_2,
                    color=dropdown_1,box=True, points="all",
                    hover_data=Xdf.columns
                    )
    fig.update_layout(
    title=str(dropdown_2)+' vs '+str(dropdown_1),
    title_x=0.5,
    xaxis_title=None,
    yaxis_title=None,
    #legend_title=str(dropdown_1)+'vs'+str(dropdown_2),
    font=dict(
        family="sans-serif",
        size=18,
        color="RebeccaPurple"
    ),
    plot_bgcolor='rgba(245, 238, 248,1.0)',
    paper_bgcolor='white'
    )
    return (fig)

#-------------------------Callback3---------------------------------
@app.callback(
    Output(component_id='graph_3', component_property='figure'),
    Input(component_id='dropdown_1', component_property='value'),
    Input(component_id='dropdown_2', component_property='value'),
    Input(component_id='radioitem_1',component_property='value')
)

def update_graph(dropdown_1 , dropdown_2,radioitem_1):
    Xdf=df
    Xdf = df.sort_values(by=dropdown_2, ascending=False)
    xbin=len(Xdf.yr_construccion.unique())
    fig = px.histogram(Xdf,x=radioitem_1,y=dropdown_2,color=dropdown_1,
              #color_discrete_sequence=px.colors.qualitative.Set1,
              histfunc='avg',nbins=xbin, barmode='group'
              )
    fig.update_layout(
    title='Valor Medio de ' + str(dropdown_2) +' vs '+ str(dropdown_1),
    title_x=0.5,
    xaxis_title=None,
    yaxis_title=None,
    legend_title=str(dropdown_1),
    font=dict(
        family="sans-serif",
        size=18,
        color="RebeccaPurple"
    ),
    plot_bgcolor='rgba(245, 238, 248,1.0)',
    paper_bgcolor='white'
    )

    return (fig)

#-------------------------Callback 4 mapa de calor-----------------
@app.callback(
    Output(component_id='graph_4', component_property='figure'),
    Input(component_id='radioitem_2', component_property='value')
)

def update_graph(radioitem_2):
    dfx=dic[radioitem_2][0]
    dfy=dic[radioitem_2][1]
    fig = px.scatter(df, y=dfy, x=dfx, color='Precio',size='Precio'#,symbol='precio'
                     )

    fig.update_layout(
    title= str(dfy) +' vs '+ str(dfx),
    title_x=0.5,
    xaxis_title=None,
    yaxis_title=None,
    #legend_title=str(Precio),
    font=dict(
        family="sans-serif",
        size=18,
        color="RebeccaPurple"
    ),
    plot_bgcolor='rgba(245, 238, 248,1.0)',
    paper_bgcolor='white'
    )
    return (fig)


#--------------------------CALLBACK 5-----------------------
@app.callback(
    Output(component_id='graph_5', component_property='figure'),
    Input(component_id='id_hab', component_property='value'),
    Input(component_id='id_grade', component_property='value'),
    Input(component_id='id_baños', component_property='value'),
    Input(component_id='id_con', component_property='value'),
    Input(component_id='id_mar', component_property='value') ,
    Input(component_id='id_input1', component_property='value'),
    Input(component_id='id_input2', component_property='value')
)


def update_graph(id_hab,id_grade,id_baños,id_con,id_mar,id_input1,id_input2):


    Xdf1=df[df['no_habitaciones'].isin(id_hab)]

    Xdf2=Xdf1[Xdf1['Calidad'].isin(id_grade)]

    Xdf3=Xdf2[Xdf2['no_baños'].isin(id_baños)]

    Xdf4=Xdf3[Xdf3['Condicion'].isin(id_con)]

    Xdf5=Xdf4[Xdf4['Vistaalmar'].isin(id_mar)]

    Xdf6=Xdf5[ (Xdf5['Precio'] >= id_input1) & (Xdf5['Precio'] <= id_input2)]

    fig=px.scatter_mapbox(Xdf6,lat='Latitud', lon='Longitud',
                      size='Precio', color='Precio',
                      color_continuous_scale= px.colors.sequential.Plasma,
                      size_max= 8,
                      zoom=8,
                      mapbox_style='open-street-map',
                      hover_data=[ 'Precio','id', 'm2_habitables', 'm2_Tot', 'Latitud', 'Longitud'],
                      #hover_data=['Precio',"m2_habitables","m2_Tot",'Latitud','Longitud']
                      )


    return (fig)

In [9]:
df.columns

Index(['Unnamed: 0', 'id', 'Fecha', 'Precio', 'no_habitaciones', 'no_baños',
       'Habitacion_baño', 'm2_habitables', 'm2_Tot', 'no_pisos', 'Vistaalmar',
       'Indice_vista', 'Condicion', 'Calidad', 'm2_snm', 'm2_bnm',
       'yr_construccion', 'CP', 'Latitud', 'Longitud', 'm2_15viviendas',
       'm2_15lotes'],
      dtype='object')

In [10]:
#==================RUN=============
if __name__=='__main__':
    app.run_server(debug=False,port=8000,mode='inline'
                   #,host='0.0.0.0',
                  )

<IPython.core.display.Javascript object>